## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units

In [ ]:
# Import my library:

import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import snapshot
import simulation
import dataset_comp

In [ ]:
import importlib
importlib.reload(snapshot)
importlib.reload(dataset_comp)
importlib.reload(simulation)

# Rotation Curves of Dwarf Galaxies

## Motivation

I plot bundles of rotation curves of satellites and isolated galaxies, binning them by mass. I expect that the curvaton rotation curves are "shallower" around the centre. In addition, I try to see if there is difference in the diversity of density profiles between the models, or if the difference is simply a characteristic shift towards shallower profiles, as found in Lovell (2016) for the sterile neutrino model.

---

## Set Parameters for the Plots

Choose the snapshot and the simulations, and define M31 and MW in each simulation. Also, set the colors used for each simulation:

In [ ]:
snap_id = 127
sim_ids = ['V1_MR_fix', 'V1_MR_curvaton_p082_fix']
names = ['LCDM', 'p082']
colors = ['black', 'red']

m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = 'by_r'
maxdi = 2000 # Maximum distance from LG centre for isolated

Set lower limit for the value of $v_\mathrm{max}$ of the galaxies to be included (separately for satellites and isolated galaxies):

In [ ]:
sat_down = 10
isol_down = 15

---

## Retrieve Data

### Create a Dictionary

For easy handling of the relevant data, define a data dictionary that, at the top level, has entries for all simulations. Under each simulation entry, add items for the needed datasets and, under the 'Selections' key, a sub-dictionary of masking arrays for each needed condition (e.g. satellite, luminous, $v_\mathrm{max}$ inside range, etc.).

First, add the above definitions into the data dict:

In [ ]:
data = {}
for name, sim_id, m31_ns, mw_ns, col in zip(names, sim_ids, m31, mw, colors):
    data[name] = {'snapshot': snapshot.Snapshot(sim_id, snap_id, name=name),
                  'M31_identifier': m31_ns,
                  'MW_identifier': mw_ns,
                  'Color': col}

Compute the mass bins for satellites and isolated galaxies. I use four bins of equal width in dex. The range used for binning is from <tt>sat_down</tt> (or <tt>isol_down</tt>) 

In [ ]:
sat_up = 0
isol_up = 0

for name, sim_data in data.items():
    # Get data:
    snap = sim_data['snapshot']
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    
    # Split into satellites:
    if distinction == 'by_r':
        masks_sat, mask_isol = dataset_comp.split_satellites_by_distance(
            snap, sim_data['M31_identifier'], sim_data['MW_identifier'], max_dist_isol=maxdi)
    elif distinction == 'by_gn':
        masks_sat, mask_isol = dataset_comp.split_satellites_by_group_number(
            snap, sim_data['M31_identifier'], sim_data['MW_identifier'])
    
    # Update upper mass limits:
    sat_up = max(sat_up, np.max(vmax[np.logical_or.reduce(masks_sat)]))
    isol_up = max(isol_up, np.max(vmax[mask_isol]))

# To include the most massive galaxy inside the bin:
sat_up += sat_up * 0.001 
isol_up += isol_up * 0.001 

# Compute satellite bins:
down = np.log10(sat_down); up = np.log10(sat_up)
dx = (up - down) / 4
print(dx)
binc = np.array([(down + dx/2) + dx*n for n in range(4)])
dex_bins = np.array([[c-dx/2,c,c+dx/2] for c in binc])
sat_bins = 10**dex_bins
print(sat_bins)

# Compute isolated galaxy bins:
down = np.log10(isol_down); up = np.log10(isol_up)
dx = (up - down) / 4
print(dx)
binc = np.array([(down + dx/2) + dx*n for n in range(4)])
dex_bins = np.array([[c-dx/2,c,c+dx/2] for c in binc])
isol_bins = 10**dex_bins
print(isol_bins)

Then, loop over simulations, retrieve data, compute masking arrays, and add to the dictionary:

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data['snapshot']
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    rmax = max_point[:,1] * units.cm.to(units.kpc)
    
    # Retrieve rotation curves and split by subhalo:
    rot_curves = snap.get_subhalos(
        'Vcirc', group='Extended/RotationCurve/All')
    sub_offset = snap.get_subhalos(
        'SubOffset', group='Extended/RotationCurve/All')
    v_circ = rot_curves[:,0] * units.cm.to(units.km)
    radii = rot_curves[:,1] * units.cm.to(units.kpc)
    v_circ = np.array(np.split(v_circ, sub_offset[1:]))
    radii = np.array(np.split(radii, sub_offset[1:]))
    
    # Split into satellites:
    if distinction == 'by_r':
        masks_sat, mask_isol = dataset_comp.split_satellites_by_distance(
            snap, sim_data['M31_identifier'], sim_data['MW_identifier'], max_dist_isol=maxdi)
    elif distinction == 'by_gn':
        masks_sat, mask_isol = dataset_comp.split_satellites_by_group_number(
            snap, sim_data['M31_identifier'], sim_data['MW_identifier'])
    
    # Compute masks for vmax bins:
    sat_bin_masks = [dataset_comp.prune_vmax(snap, low_lim=b[0], up_lim=b[2])
                     for b in sat_bins]
    isol_bin_masks = [dataset_comp.prune_vmax(snap, low_lim=b[0], up_lim=b[2])
                     for b in isol_bins]

    # Compute other masking arrays:
    mask_lum, mask_dark = dataset_comp.split_luminous(snap)
    mask_shallow = rmax > 10
    
    # Add selections (masking arrays):
    data[name]['Selections'] = {
        'M31': masks_sat[0],
        'MW': masks_sat[1],
        'Satellite': np.logical_or.reduce(masks_sat),
        'Isolated': mask_isol,
        'VmaxBins': {
            'Satellite': {
                'Low': sat_bin_masks[0], 
                'SemiLow': sat_bin_masks[1],
                'SemiHigh': sat_bin_masks[2],
                'High': sat_bin_masks[3]  
            },
            'Isolated': {
                'Low': isol_bin_masks[0], 
                'SemiLow': isol_bin_masks[1],
                'SemiHigh': isol_bin_masks[2],
                'High': isol_bin_masks[3]  
            }
        },
        'Luminous': mask_lum,
        'Dark': mask_dark,
        'Shallow': mask_shallow
    }
    
    # Add datasets to dictionary:
    data[name]['Vmax'] = vmax
    data[name]['Rmax'] = rmax
    data[name]['Vcirc'] = v_circ
    data[name]['Radius'] = radii

---

In [ ]:
# Line transparency:
a = 0.7

## Plot Unmodified Rotation Curve Bundles

### Plot Satellites

In [ ]:
# Store the figure into a variable ("plain": unmodified rot. curves), for saving etc.:
fig_sat_plain, axes = plt.subplots(nrows=2, ncols=2, sharex='row', sharey='row', figsize=(8,8))

plt.subplots_adjust(wspace=0.1)
plt.subplots_adjust(hspace=0.1)

# Set axis limits:
axes[0,0].set_xlim(0,10)
axes[0,0].set_ylim(0,50)
axes[1,0].set_xlim(0,30)
axes[1,0].set_ylim(0,90)

# Set labels:
s=14
axes[1,0].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[1,1].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[0,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)
axes[1,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)

fig_sat_plain.suptitle('Satellite Galaxies', y=1.01, fontsize=s)

plt.tight_layout()

Plot dark:

In [ ]:
dark_col = ['gray', 'pink']; b = 0.5

# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['Low'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask] + 20
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=dark_col[i], alpha=b)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiLow'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask] + 20
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=dark_col[i], alpha=b)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiHigh'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=dark_col[i], alpha=b)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['High'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=dark_col[i], alpha=b)

fig_sat_plain

Plot luminous:

In [ ]:
# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=entry['Color'], alpha=a)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=entry['Color'], alpha=a)

fig_sat_plain

In [ ]:
axes[0,0].text(0.1, 0.9,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[0,1], sat_bins[0,2] - sat_bins[0,1], 
                   sat_bins[0,1] - sat_bins[0,0]),
               transform=axes[0,0].transAxes, 
               horizontalalignment='left',
               verticalalignment='top',
               fontsize=s)
axes[0,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[1,1], sat_bins[1,2] - sat_bins[1,1], 
                   sat_bins[1,1] - sat_bins[1,0]),
               transform=axes[0,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[2,1], sat_bins[2,2] - sat_bins[2,1], 
                   sat_bins[2,1] - sat_bins[2,0]),
               transform=axes[1,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[3,1], sat_bins[3,2] - sat_bins[3,1], 
                   sat_bins[3,1] - sat_bins[3,0]),
               transform=axes[1,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)

fig_sat_plain

### Plot Isolated Galaxies

In [ ]:
# Store the figure into a variable ("plain": unmodified rot. curves), for saving etc.:
fig_isol_plain, axes = plt.subplots(nrows=2, ncols=2, sharex='row', sharey='row', figsize=(8,8))

plt.subplots_adjust(wspace=0.1)
plt.subplots_adjust(hspace=0.1)

# Set axis limits:
axes[0,0].set_xlim(0,10)
axes[0,0].set_ylim(0,50)
axes[1,0].set_xlim(0,30)
axes[1,0].set_ylim(0,90)

# Set labels:
s=14
axes[1,0].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[1,1].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[0,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)
axes[1,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)

fig_isol_plain.suptitle('Isolated Galaxies', y=1.01, fontsize=s)

plt.tight_layout()

Plot dark:

In [ ]:
dark_col = ['gray', 'pink']; b = 0.5

# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['Low'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask] + 20
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=dark_col[i], alpha=b)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiLow'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask] + 20
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=dark_col[i], alpha=b)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiHigh'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=dark_col[i], alpha=b)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['High'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=dark_col[i], alpha=b)

fig_isol_plain

In [ ]:
# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=entry['Color'], alpha=a)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=entry['Color'], alpha=a)

fig_isol_plain

In [ ]:
axes[0,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[0,1], isol_bins[0,2] - isol_bins[0,1], 
                   isol_bins[0,1] - isol_bins[0,0]),
               transform=axes[0,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[0,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[1,1], isol_bins[1,2] - isol_bins[1,1], 
                   isol_bins[1,1] - isol_bins[1,0]),
               transform=axes[0,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[2,1], isol_bins[2,2] - isol_bins[2,1], 
                   isol_bins[2,1] - isol_bins[2,0]),
               transform=axes[1,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[3,1], isol_bins[3,2] - isol_bins[3,1], 
                   isol_bins[3,1] - isol_bins[3,0]),
               transform=axes[1,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)

fig_isol_plain

### Save the Figures

In [ ]:
filename = 'sat_rotcurves_plain'
for name in names:
    filename += '_{}'.format(name)
filename += '.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig_sat_plain.savefig(filename, dpi=300, bbox_inches='tight')

In [ ]:
filename = 'isol_rotcurves_plain'
for name in names:
    filename += '_{}'.format(name)
filename += '.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig_isol_plain.savefig(filename, dpi=300, bbox_inches='tight')

### Highlight Shallow Ones

Select "shallow" profiled satellite galaxies and highlight them. This is to inspect the usefulness of the different plotting schemes.

In [ ]:
axes = np.array(fig_sat_plain.get_axes()).reshape((2,2))
shallow_col = ['blue', 'lightgreen']

for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=shallow_col[i])
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=shallow_col[i])
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=shallow_col[i])
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=shallow_col[i])

fig_sat_plain

---

## Plot Scaled Rotation Curves

### Satellite Galaxies

In [ ]:
# Store the figure into a variable:
fig_sat_scaled, axes = plt.subplots(nrows=2, ncols=2, sharex='row', sharey='row', figsize=(8,8))

plt.subplots_adjust(wspace=0.1)
plt.subplots_adjust(hspace=0.1)

# Set axis limits:
axes[0,0].set_xlim(0, 1.7)
axes[1,0].set_xlim(0, 1.7)
axes[0,0].set_ylim(0, 1.5)
axes[1,0].set_ylim(0, 1.5)

# Set labels:
s=14
axes[1,0].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[1,1].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[0,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)
axes[1,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)

fig_sat_scaled.suptitle('Satellite Galaxies', y=1.01, fontsize=s)

# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[0,0].plot(r/rm, v/vm, c=entry['Color'], alpha=a)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[0,1].plot(r/rm, v/vm, c=entry['Color'], alpha=a)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[1,0].plot(r/rm, v/vm, c=entry['Color'], alpha=a)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[1,1].plot(r/rm, v/vm, c=entry['Color'], alpha=a)
        
plt.tight_layout()

Add bin ranges to the figure explicitly:

In [ ]:
axes[0,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[0,1], sat_bins[0,2] - sat_bins[0,1], 
                   sat_bins[0,1] - sat_bins[0,0]),
               transform=axes[0,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[0,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[1,1], sat_bins[1,2] - sat_bins[1,1], 
                   sat_bins[1,1] - sat_bins[1,0]),
               transform=axes[0,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[2,1], sat_bins[2,2] - sat_bins[2,1], 
                   sat_bins[2,1] - sat_bins[2,0]),
               transform=axes[1,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[3,1], sat_bins[3,2] - sat_bins[3,1], 
                   sat_bins[3,1] - sat_bins[3,0]),
               transform=axes[1,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)

fig_sat_scaled

### Isolated Galaxies

In [ ]:
# Store the figure into a variable:
fig_isol_scaled, axes = plt.subplots(nrows=2, ncols=2, sharex='row', sharey='row', figsize=(8,8))

plt.subplots_adjust(wspace=0.1)
plt.subplots_adjust(hspace=0.1)

# Set axis limits:
axes[0,0].set_xlim(0, 1.7)
axes[1,0].set_xlim(0, 1.7)
axes[0,0].set_ylim(0, 1.5)
axes[1,0].set_ylim(0, 1.5)

# Set labels:
s=14
axes[1,0].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[1,1].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[0,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)
axes[1,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)

fig_isol_scaled.suptitle('Isolated Galaxies', y=1.01, fontsize=s)

# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[0,0].plot(r/rm, v/vm, c=entry['Color'], alpha=a)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[0,1].plot(r/rm, v/vm, c=entry['Color'], alpha=a)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[1,0].plot(r/rm, v/vm, c=entry['Color'], alpha=a)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[1,1].plot(r/rm, v/vm, c=entry['Color'], alpha=a)
        
plt.tight_layout()

Add bin ranges to the figure explicitly:

In [ ]:
axes[0,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[0,1], isol_bins[0,2] - isol_bins[0,1], 
                   isol_bins[0,1] - isol_bins[0,0]),
               transform=axes[0,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[0,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[1,1], isol_bins[1,2] - isol_bins[1,1], 
                   isol_bins[1,1] - isol_bins[1,0]),
               transform=axes[0,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[2,1], isol_bins[2,2] - isol_bins[2,1], 
                   isol_bins[2,1] - isol_bins[2,0]),
               transform=axes[1,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[3,1], isol_bins[3,2] - isol_bins[3,1], 
                   isol_bins[3,1] - isol_bins[3,0]),
               transform=axes[1,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)

fig_isol_scaled

### Save the Figures

In [ ]:
filename = 'sat_rotcurves_scaled'
for name in names:
    filename += '_{}'.format(name)
filename += '.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig_sat_scaled.savefig(filename, dpi=300, bbox_inches='tight')

In [ ]:
filename = 'isol_rotcurves_scaled'
for name in names:
    filename += '_{}'.format(name)
filename += '.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig_isol_scaled.savefig(filename, dpi=300, bbox_inches='tight')

### Highlight Shallow Ones

In [ ]:
axes = np.array(fig_sat_scaled.get_axes()).reshape((2,2))
shallow_col = ['blue', 'lightgreen']

for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[0,0].plot(r/rm, v/vm, c=shallow_col[i])
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[0,1].plot(r/rm, v/vm, c=shallow_col[i])
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[1,0].plot(r/rm, v/vm, c=shallow_col[i])
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    vmax = entry['Vmax'][mask]
    rmax = entry['Rmax'][mask]
    for v, r, vm, rm in zip(v_circ, radius, vmax, rmax):
        axes[1,1].plot(r/rm, v/vm, c=shallow_col[i])

fig_sat_scaled

---

## Plot Logarithmic Rotation Curves

### Satellite Galaxies

In [ ]:
# Store the figure into a variable:
fig_sat_log, axes = plt.subplots(nrows=2, ncols=2, sharex='row', sharey='row', figsize=(8,8))

plt.subplots_adjust(wspace=0.1)
plt.subplots_adjust(hspace=0.1)

# Set logarithmic axes:
axes[0,0].set_xscale('log')
axes[0,0].set_yscale('log')
axes[0,1].set_xscale('log')
axes[0,1].set_yscale('log')
axes[1,0].set_xscale('log')
axes[1,0].set_yscale('log')
axes[1,1].set_xscale('log')
axes[1,1].set_yscale('log')

# Set axis limits:
axes[0,0].set_xlim(0.3,10)
axes[0,0].set_ylim(6,40)
axes[1,0].set_xlim(0.3,30)
axes[1,0].set_ylim(6,90)

# Set labels:
s=14
axes[1,0].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[1,1].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[0,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)
axes[1,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)

fig_sat_log.suptitle('Satellite Galaxies', y=1.01, fontsize=s)

# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=entry['Color'], alpha=a)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=entry['Color'], alpha=a)

plt.tight_layout()

Add bin ranges to the figure explicitly:

In [ ]:
axes[0,0].text(0.1, 0.9,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[0,1], sat_bins[0,2] - sat_bins[0,1], 
                   sat_bins[0,1] - sat_bins[0,0]),
               transform=axes[0,0].transAxes, 
               horizontalalignment='left',
               verticalalignment='top',
               fontsize=s)
axes[0,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[1,1], sat_bins[1,2] - sat_bins[1,1], 
                   sat_bins[1,1] - sat_bins[1,0]),
               transform=axes[0,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[2,1], sat_bins[2,2] - sat_bins[2,1], 
                   sat_bins[2,1] - sat_bins[2,0]),
               transform=axes[1,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[3,1], sat_bins[3,2] - sat_bins[3,1], 
                   sat_bins[3,1] - sat_bins[3,0]),
               transform=axes[1,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)

fig_sat_log

### Isolated Galaxies

In [ ]:
# Store the figure into a variable:
fig_isol_log, axes = plt.subplots(nrows=2, ncols=2, sharex='row', sharey='row', figsize=(8,8))

plt.subplots_adjust(wspace=0.1)
plt.subplots_adjust(hspace=0.1)

# Set logarithmic axes:
axes[0,0].set_xscale('log')
axes[0,0].set_yscale('log')
axes[0,1].set_xscale('log')
axes[0,1].set_yscale('log')
axes[1,0].set_xscale('log')
axes[1,0].set_yscale('log')
axes[1,1].set_xscale('log')
axes[1,1].set_yscale('log')

# Set axis limits:
axes[0,0].set_xlim(0.3,10)
axes[0,0].set_ylim(6,40)
axes[1,0].set_xlim(0.3,30)
axes[1,0].set_ylim(6,90)

# Set labels:
s=14
axes[1,0].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[1,1].set_xlabel('$r[\mathrm{kpc}]$', fontsize=s)
axes[0,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)
axes[1,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=s)

fig_isol_log.suptitle('Isolated Galaxies', y=1.01, fontsize=s)

# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=entry['Color'], alpha=a)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=entry['Color'], alpha=a)

plt.tight_layout()

Add bin ranges to the figure explicitly:

In [ ]:
axes[0,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[0,1], isol_bins[0,2] - isol_bins[0,1], 
                   isol_bins[0,1] - isol_bins[0,0]),
               transform=axes[0,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[0,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[1,1], isol_bins[1,2] - isol_bins[1,1], 
                   isol_bins[1,1] - isol_bins[1,0]),
               transform=axes[0,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[2,1], isol_bins[2,2] - isol_bins[2,1], 
                   isol_bins[2,1] - isol_bins[2,0]),
               transform=axes[1,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[3,1], isol_bins[3,2] - isol_bins[3,1], 
                   isol_bins[3,1] - isol_bins[3,0]),
               transform=axes[1,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)

fig_isol_log

### Save the Figures

In [ ]:
filename = 'sat_rotcurves_log'
for name in names:
    filename += '_{}'.format(name)
filename += '.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig_sat_log.savefig(filename, dpi=300, bbox_inches='tight')

In [ ]:
filename = 'isol_rotcurves_log'
for name in names:
    filename += '_{}'.format(name)
filename += '.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig_isol_log.savefig(filename, dpi=300, bbox_inches='tight')

### Highlight Shallow

In [ ]:
axes = np.array(fig_sat_log.get_axes()).reshape((2,2))
shallow_col = ['blue', 'lightgreen']

for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=shallow_col[i])
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=shallow_col[i])
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=shallow_col[i])
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['Shallow'],
                                  entry['Selections']['VmaxBins']['Satellite']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=shallow_col[i])

fig_sat_log